In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import scipy

In [4]:
from sonic_screwdrivers import DFtoArrayTransformer
from sonic_screwdrivers.smart_matchers.bipartite_graph_matching import (
    optimize_bipartite_graph_matches,
    match_with_min_costs,
    transform_matches_to_df
)



ValueError: Attempted relative import in non-package

In [4]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

Test nans
===

In [ ]:
cost = np.array([[np.inf, np.inf, 2, 0], [np.nan, np.inf, 0, 0], [0, np.inf, np.inf, 0]])

In [ ]:
cost

In [ ]:
cost[cost < np.inf].max()

In [ ]:
cost[(cost == np.inf) | (cost==2)] = 5

In [ ]:
cost

In [ ]:
np.NaN

In [5]:
cost_df = pd.DataFrame(np.array([[np.inf, np.nan, 0, 0], [np.nan, np.inf, 0, 0], [0, np.inf, np.inf, 0]]))


In [6]:
find_bipartite_graph_matches(cost_df)


,0,1,2,3
0,NaN,NaN,1.0,0.0
1,NaN,NaN,0.0,1.0
2,1.0,NaN,NaN,0.0


In [ ]:
cost_array = np.array([[np.inf, np.nan, 0, 0], [np.nan, np.inf, 0, 0], [0, np.inf, np.inf, 0]])

In [ ]:
num_rows, num_cols = cost_array.shape

In [ ]:
max_dim = max(num_rows, num_cols)


In [ ]:
cost_array_ = np.full((max_dim, max_dim), np.inf)

In [ ]:
cost_array_[0:num_rows, 0:num_cols] = cost_array

In [ ]:
assert cost_array_.shape[0] == cost_array_.shape[1]

In [ ]:
try:
    practicalInfinity = 2 * cost_array[cost_array < np.inf].max() + 1
except ValueError:
    practicalInfinity = 1

In [ ]:
cost_array_[(cost_array_ == np.inf) | (cost_array_ == np.nan)] = practicalInfinity

In [ ]:
practicalInfinity

In [ ]:
np.isnan(cost_array_[1, 0])

In [ ]:
cost_array_[(cost_array_ == np.nan)]

In [ ]:
cost_array_[(cost_array_ == np.inf) | (cost_array_ == np.nan)]

In [ ]:
iSet, jSet = scipy.optimize.linear_sum_assignment(cost_array_)

In [ ]:

# Make the matrix square
cost_array_ = np.full((max_dim, max_dim), np.inf)
cost_array_[0:num_rows, 0:num_cols] = cost_array
assert cost_array_.shape[0] == cost_array_.shape[1]

# Replace infinity with a reasonable value
try:
    practicalInfinity = 2 * cost_array[cost_array < np.inf].max() + 1
except ValueError:
    practicalInfinity = 1

cost_array_[(cost_array_ == np.inf) | (cost_array_ == np.nan)] = practicalInfinity

In [ ]:
match_with_min_weights(cost)

Test a very small array
===

In [ ]:
cost = np.array([[np.inf, np.inf, 0, 0], [np.inf, np.inf, 0, 0], [0, np.inf, np.inf, 0]])

In [ ]:
cost

In [ ]:
match_with_min_weights(cost)

In [ ]:
cost_df = pd.DataFrame(cost)

In [ ]:
%time find_bipartite_graph_matches(cost_df)

In [ ]:
actual = find_bipartite_graph_matches(cost_df)

In [ ]:
expected = pd.DataFrame([[np.NaN, np.NaN, 1.0, 0], [np.NaN, np.NaN, 0, 1.0], [1.0, np.NaN, np.NaN, 0]])


In [ ]:
actual.dtypes

In [ ]:
expected.dtypes

In [ ]:
actual

In [ ]:
expected

Test an array with no matches
===

In [ ]:
cost = np.array([[np.inf, np.inf, np.inf, np.inf], [np.inf, np.inf, np.inf, np.inf], [np.inf, np.inf, np.inf, np.inf]])

In [ ]:
cost_df = pd.DataFrame(cost)

In [ ]:
actual = find_bipartite_graph_matches(cost_df)

In [ ]:
actual

In [ ]:
match_with_min_weights(cost)

In [ ]:
cost

Test a larger array
===

In [ ]:
rand_cost = np.random.rand(1000, 3100)

In [ ]:
rand_cost_df = pd.DataFrame(rand_cost)

In [ ]:
rand_cost.shape

In [ ]:
%time rand_cost_mastches = find_bipartite_graph_matches(rand_cost_df)

In [ ]:
rand_cost = np.random.rand(100, 61)

In [ ]:
indices= list(np.ndindex(rand_cost.shape))

In [ ]:
np.random.shuffle(indices)

In [ ]:
for i, index in enumerate(indices):
    if i % 5 == 0:
        rand_cost[index] = np.inf
    else:
        if rand_cost[index] > 0.5:
            rand_cost[index] = 1
        elif rand_cost[index] < 0.1:
            rand_cost[index] = 0

In [ ]:
rand_cost

In [ ]:
rand_cost_df = pd.DataFrame(rand_cost)

In [ ]:
rand_cost.shape

In [ ]:
%timeit rand_cost_matches = find_bipartite_graph_matches(rand_cost_df)

In [ ]:
rand_cost_matches  = find_bipartite_graph_matches(rand_cost_df)

In [ ]:
rand_cost_df

In [ ]:
rand_cost_matches